In [ ]:
import numpy as np
import time
import random
import scipy
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6, floatmode='fixed')
import random

In [ ]:
def createMtr(num, n=0):

    if num == 1:
      A = np.zeros((n, n))
      for i in range(len(A)):
          for j in range(len(A[i])):
              A[i, j] = i + j + 1

    elif num == 2:
        A = np.array([[5, 6, 3],
                      [-1, 0, 1],
                      [1, 2, -1]])

    elif num == 3:
        A = np.array([[-1, -6],
                      [2, 7]])

    elif num == 4:
        A = np.array([[-3, -4],
                      [2, 2]])

    elif num == 5:
        A = np.zeros((n, n))
        for i in range(len(A)):
          for j in range(len(A[i])):
              A[i, j] = random.uniform(-100, 100)


    return A



A = createMtr(5, 100)
#print('Матрица A')
#print(A)

# QR-алгоритм для матрицы в обычной форме

In [ ]:
#сумма квадратов элементов ниже главной диагонали
def sumSqBelow(A):
    sum = 0
    n, m = np.shape(A)
    rows, cols = np.tril_indices(n, -1, m)
    for (row, col) in zip(rows, cols):
         sum += (A[row, col])**2

    return sum


def norm(a, b):
    sum = 0
    a = np.sort(a)
    b = np.sort(b)
    for i in range(len(a)):
        sum += (a[i]-b[i])**2
    return np.sqrt(sum)

#print(sumSqBelow(A))

def houseVector(a):
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)

    return v,tau


def Householder(A: np.ndarray):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m)

    for j in range(0, n):
        v, tau = houseVector(R[j:, j, np.newaxis])

        H = np.identity(m)
        H[j:, j:] -= tau * (v @ v.T)
        R = H @ R
        Q = H @ Q

    return Q[:n].T, np.triu(R[:n])



#QR-алгоритм
#def QRalg(A):
  #  A1 = np.copy(A)
 #   start_time = time.time()
  #  while True:
   #     Q, A1 = np.linalg.qr(A1)
     #   A1 = A1 @ Q
    #    if sumSqBelow(A1) < 10**(-12):
     #      break
   #end_time = time.time()
   # return A1, end_time - start_time


def QRalg(A):
	eps = 10**(-12); cnttol = 500
	m = A.shape[0]; eig = np.zeros(m, dtype=complex)
	n = m - 1

	while n > 0:
		cnt = 0

		while max(abs(A[n, 0:n])) > eps and cnt < cnttol:
			cnt += 1
			Q, R = np.linalg.qr(A)
			A = R @ Q

		if cnt < cnttol:
			eig[n]=A[n, n]
			n -= 1
			A = A[:n+1, :n+1]

		else:
			disc = complex(( A[-2, -2] - A[-1, -1] )**2 + 4*A[-1, -2]*A[-2, -1])
			eig[n]=( A[-2, -2] + A[-1, -1] + np.sqrt(disc) )/2
			eig[n-1]=( A[-2, -2] + A[-1, -1] - np.sqrt(disc) )/2
			n -= 2
			A = A[:n+1, :n+1]

	if n==0:
		eig[0] = A[0,0]

	return eig

#print('Собственные числа матрицы А с помощью втроенной функции: ')
a, b = np.linalg.eig(A)
#print(a, '\n')
s = time.time()
eig = QRalg(A)
en = time.time()
#print('Матрица после QR-алгоритма:')
#print(A1)
#print('Собственные числа через QR-алгоритм: ')
#print(eig, '\n')

print('Норма разности:', norm(a, eig), '\n')

print('Время, затраченное на QR-алгоритм:', en-s)




# QR-алгоритм для матрицы в форме Хессенберга

In [ ]:
#функция создания орта
def ort(size, pos):
    if size == 1:
       return 1
    else:
       e = np.zeros(size)
       e[pos] = 1
       return e

#приведение к форме Хессенберга
def hessenberg(A1):
    A = np.copy(A1)

    if (len(A) <= 4):
      for i in range(len(A)-2):
        x = A[i+1:, i]

        u = np.zeros(len(x))
        u = x + np.sign(x[0]) * np.linalg.norm(x, 2) * ort(len(x), 0)

        P = np.eye(len(x)) - 2 * ((np.outer(u, u.transpose()))/(np.dot(u.transpose(), u)))

        P1 = np.zeros((len(A), len(A)))
        P1[i+1:, i+1:] = P
        for i in range(len(P)):
            if P1[i, i] == 0:
               P1[i, i] = 1
            else:
              break

        A = P1.T @ A @ P1


    else:
      for i in range(len(A)-3):
        x = A[i+1:, i]

        u = np.zeros(len(x))
        u = x + np.sign(x[0]) * np.linalg.norm(x, 2) * ort(len(x), 0)

        P = np.eye(len(x)) - 2 * ((np.outer(u, u.transpose()))/(np.dot(u.transpose(), u)))

        P1 = np.zeros((len(A), len(A)))
        P1[i+1:, i+1:] = P
        for i in range(len(P)):
            if P1[i, i] == 0:
               P1[i, i] = 1
            else:
              break

        A = P1.T @ A @ P1

    return A


#print('Матрица A')
#print(A, '\n')

H1 = hessenberg(np.copy(A))
print('Матрица в форме Хессенберга')
print(H1, '\n')

print('Матрица в форме Хессенберга (встроенная функция)')
print(scipy.linalg.hessenberg(np.copy(A)))

Матрица в форме Хессенберга
[[ 1.000000 -7.348469  0.000000  0.000000  0.000000]
 [-7.348469 24.370370  3.312693  0.000000  0.000000]
 [ 0.000000  3.312693 -0.370370  0.000000 -0.000000]
 [ 0.000000 -0.000000 -0.000000  0.000000  0.000000]
 [ 0.000000 -0.000000  0.000000  0.000000 -0.000000]] 

Матрица в форме Хессенберга (встроенная функция)
[[ 1.000000 -7.348469  0.000000 -0.000000  0.000000]
 [-7.348469 24.370370  3.312693 -0.000000  0.000000]
 [ 0.000000  3.312693 -0.370370  0.000000 -0.000000]
 [ 0.000000  0.000000  0.000000 -0.000000 -0.000000]
 [ 0.000000  0.000000  0.000000  0.000000 -0.000000]]


In [ ]:
print('Собственные числа матрицы А с помощью вcтроенной функции: ')
print(a, '\n')

s = time.time()
eig = QRalg(H1)
en = time.time()
#print('Матрица Н после QR-алгоритма')
#print(H1)
print('Собственные числа через QR-алгоритм: ')
print(eig, '\n')
print('Норма разности:', norm(a, eig), '\n')
print('Время, затраченное на QR-алгоритм:', en-s)

Собственные числа матрицы А с помощью вcтроенной функции: 
[26.861407+0.000000j -1.861407+0.000000j -0.000000+0.000000j
 -0.000000-0.000000j  0.000000+0.000000j] 

Собственные числа через QR-алгоритм: 
[26.861407+0.000000j -1.861407+0.000000j -0.000000+0.000000j
  0.000000+0.000000j -0.000000+0.000000j] 

Норма разности: (3.2912231136981212e-15-1.596041105696207e-16j) 

Время, затраченное на QR-алгоритм: 0.0009055137634277344


#QR-алгоритм со сдвигами

In [ ]:
H = hessenberg(A)

def shiftedqr(A):
	eps = 10**(-12); cnttol = 500
	m = A.shape[0]; eig = np.zeros(m, dtype=complex)
	n = m - 1

	while n > 0:
		cnt = 0

		while max(abs(A[n, 0:n])) > eps and cnt < cnttol:
			cnt += 1
			mu = A[n, n]
			Q, R = np.linalg.qr( A - mu*np.eye(n+1) )
			A = R @ Q + mu*np.eye(n + 1)

		if cnt < cnttol:
			eig[n]=A[n, n]
			n -= 1
			A = A[:n+1, :n+1]

		else:
			disc = complex(( A[-2, -2] - A[-1, -1] )**2 + 4*A[-1, -2]*A[-2, -1])
			eig[n]=( A[-2, -2] + A[-1, -1] + np.sqrt(disc) )/2.
			eig[n-1]=( A[-2, -2] + A[-1, -1] - np.sqrt(disc) )/2.
			n -= 2
			A = A[:n+1, :n+1]

	if n==0:
		eig[0] = A[0,0]

	return eig

start_time = time.time()
eig = shiftedqr(H)
end_time = time.time()

print('Собственные числа матрицы А с помощью втроенной функции: ')
print(a, '\n')
print('Собственные числа через QR-алгоритм со сдвигами: ')
print(eig, '\n')
print('Норма разности:', norm(a, eig), '\n')
print('Время, затраченное на QR-алгоритм со сдвигами:', end_time-start_time)


Собственные числа матрицы А с помощью втроенной функции: 
[26.861407+0.000000j -1.861407+0.000000j -0.000000+0.000000j
 -0.000000-0.000000j  0.000000+0.000000j] 

Собственные числа через QR-алгоритм со сдвигами: 
[26.861407+0.000000j -1.861407+0.000000j -0.000000+0.000000j
  0.000000+0.000000j -0.000000+0.000000j] 

Норма разности: (4.86001710681875e-29-4.987586372745744e-31j) 

Время, затраченное на QR-алгоритм со сдвигами: 0.0007712841033935547
